In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
from PIL import Image
import keras
from keras.models import Model, Sequential
from keras.utils import np_utils
from tensorflow.keras import optimizers
import tensorflow as tf
from tensorflow.keras import applications
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras import applications
from keras.layers import Add, Dense, Activation, Dropout, Flatten, Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Activation, concatenate, BatchNormalization, Concatenate
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt

In [ ]:
path = "/kaggle/input/utkfacefull/"

In [ ]:
images = np.load(path+'images.npy')
ages = np.load(path+'ages.npy')
genders = np.load(path+'genders.npy')
races = np.load(path+'races.npy')

In [ ]:
images_train, images_test, ages_train, ages_test = train_test_split(images, ages,
                                                    test_size=0.3, 
                                                    random_state=42, 
                                                    shuffle=True)
images_train, images_test, genders_train, genders_test = train_test_split(images, genders,
                                                    test_size=0.3, 
                                                    random_state=42, 
                                                    shuffle=True)
images_train, images_test, races_train, races_test = train_test_split(images, races,
                                                    test_size=0.3, 
                                                    random_state=42, 
                                                    shuffle=True)

In [ ]:
plt.imshow(images_test[0][:,:,::-1]), ages_test[0], genders_test[0], races_test[0]

In [ ]:
#data argument
train_datagen = ImageDataGenerator(
    rotation_range=45,
    zoom_range = [0.8,1.25],
    horizontal_flip = True)

In [ ]:
early_stopping = EarlyStopping( patience=7, # wait for 7 epochs
                                min_delta = 0.01, # if in 7 epochs the loss function doesn't inrease (for accuracy) 
                                               # or decrease (for val_loss) by 1%, then stop
                                verbose=1, # print the training epoch on which training was stopped
                                mode = 'min',
                                monitor='val_loss')
reduce_learning_rate = ReduceLROnPlateau(
                                monitor="val_loss",
                                patience=5, # if val_loss plateaus for 3 epochs such that it doesn't see 
                                            # an improvement of size = epsilon
                                episilon= 0.01,
                                factor=0.1,  # then we reduce the learning rate by a factor of 0.1
                                cooldown = 4, # and we wait for 4 epochs before we restart again
                                verbose=1)
# hyperparameters
lr = 0.01
epochs = 50 # setting it to a low number since this is for Kaggle, ideally you should set this to a higher number ~ 100 so that the model overfits training data, and then apply the elbow methods to select the best params on validaion data
batch_size = 16
input_shape =[100,100]

# **Create VGG16 model as base_model**

In [ ]:
inputs = Input(shape=(input_shape[0],input_shape[1], 3))
base_model = applications.vgg16.VGG16(weights = "imagenet", include_top=False, input_tensor = inputs)
base_model_output = base_model.output

for layer in base_model.layers:
   layer.trainable = False
# base_model.summary()


# base_model_input = Input(shape=(input_shape[0],input_shape[1],3))
# base_model = Convolution2D(filters=64, kernel_size=(3,3),padding='same', activation='relu')(base_model_input)
# base_model = Convolution2D(filters=64, kernel_size=(3,3),padding='same', activation='relu')(base_model)
# base_model = MaxPooling2D((2, 2), strides=(2, 2))(base_model)

# base_model = Convolution2D(filters=128, kernel_size=(3,3),padding='same', activation='relu')(base_model)
# base_model = Convolution2D(filters=128, kernel_size=(3,3),padding='same', activation='relu')(base_model)
# base_model = MaxPooling2D((2, 2), strides=(2, 2))(base_model)

# base_model = Convolution2D(filters=256, kernel_size=(3,3),padding='same', activation='relu')(base_model)
# base_model = Convolution2D(filters=256, kernel_size=(3,3),padding='same', activation='relu')(base_model)
# base_model = Convolution2D(filters=256, kernel_size=(3,3),padding='same', activation='relu')(base_model)
# base_model = MaxPooling2D((2, 2), strides=(2, 2))(base_model)

# base_model = Convolution2D(filters=512, kernel_size=(3,3),padding='same', activation='relu')(base_model)
# base_model = Convolution2D(filters=512, kernel_size=(3,3),padding='same', activation='relu')(base_model)
# base_model = Convolution2D(filters=512, kernel_size=(3,3),padding='same', activation='relu')(base_model)
# base_model = MaxPooling2D((2, 2), strides=(2, 2))(base_model)

# base_model = Convolution2D(filters=512, kernel_size=(3,3),padding='same', activation='relu')(base_model)
# base_model = Convolution2D(filters=512, kernel_size=(3,3),padding='same', activation='relu')(base_model)
# base_model = Convolution2D(filters=512, kernel_size=(3,3),padding='same', activation='relu')(base_model)
# base_model = MaxPooling2D((2, 2), strides=(2, 2))(base_model)
# base_model = keras.models.load_model('/kaggle/input/vgg-weights/vgg_face_weights.h5')

**Train the Age model**

In [ ]:
age = Flatten()(base_model_output)
age = Dense(4096,activation="relu")(age)
age = Dropout(0.5)(age)
age = Dense(4096,activation="relu")(age)
age_output = Dense(4, activation='softmax')(age)

In [ ]:
age_model = Model(inputs=inputs, outputs=age_output)

In [ ]:
age_model.load_weights("/kaggle/input/vgg-weights/age_model.h5")

In [ ]:
age_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# age_model.fit_generator(train_datagen.flow(images_train,ages_train,
#                              batch_size = batch_size),
#                   epochs=epochs,
#                   steps_per_epoch=images_train.shape[0] // batch_size,
# #                   verbose=True,
#                   validation_data=(images_test, ages_test),
#                   callbacks=[reduce_learning_rate,
#                              early_stopping])

In [ ]:
age_model.evaluate(images_test, ages_test)

In [ ]:
# age_model.save('age_model.h5')

**Train the Gender model**

In [ ]:
gender = Flatten()(base_model_output)
gender = Dense(4096,activation="relu")(gender)
gender = Dropout(0.5)(gender)
gender = Dense(2048,activation="relu")(gender)
gender_output = Dense(2, activation='sigmoid')(gender)

In [ ]:
gender_model = Model(inputs=inputs, outputs=gender_output)

In [ ]:
gender_model.load_weights("/kaggle/input/vgg-weights/gender_model.h5")

In [ ]:
# gender_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
# gender_model.fit_generator(train_datagen.flow(images_train,genders_train,
#                              batch_size = batch_size),
#                   epochs=epochs,
#                   steps_per_epoch=images_train.shape[0] // batch_size,
# #                   verbose=True,
#                   validation_data=(images_test, genders_test),
#                   callbacks=[reduce_learning_rate,
#                              early_stopping])

In [ ]:
gender_model.evaluate(images_test, genders_test)

In [ ]:
# gender_model.save('gender_model.h5')

**Train the Race model**

In [ ]:
race = Flatten()(base_model_output)
race = Dense(4096,activation="relu")(race)
race = Dropout(0.5)(race)
race = Dense(2048,activation="relu")(race)
race_output = Dense(5, activation='softmax')(race)

In [ ]:
race_model = Model(inputs=inputs, outputs=race_output)

In [ ]:
race_model.load_weights("/kaggle/input/vgg-weights/race_model.h5")

In [ ]:
# race_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# race_model.fit_generator(train_datagen.flow(images_train,races_train,
#                              batch_size = batch_size),
#                   epochs=epochs,
#                   steps_per_epoch=images_train.shape[0] // batch_size,
# #                   verbose=True,
#                   validation_data=(images_test, races_test),
#                   callbacks=[reduce_learning_rate,
#                              early_stopping])

In [ ]:
race_model.evaluate(images_test, races_test)

In [ ]:
# race_model.save('race_model.h5')

**Create combined model**

In [ ]:
for layer in age_model.layers:
    layer.trainable = False
for layer in gender_model.layers:
    layer.trainable = False
for layer in race_model.layers:
    layer.trainable = False

In [ ]:
# The combined model
combined_model_input = Input(shape=(input_shape[0],input_shape[1], 3))
age_predict = age_model(combined_model_input)
gender_predict = gender_model(combined_model_input)
race_predict = race_model(combined_model_input)
final = concatenate([age_predict,gender_predict,race_predict])
final = Dense(1024, activation = "relu")(final)
final = Dropout(0.3)(final)
final_output = Dense(4, activation = "softmax")(final)

In [ ]:
final = Model(inputs=combined_model_input, outputs=final_output)

In [ ]:
final.load_weights("/kaggle/input/vgg-weights/model.h5")

In [ ]:
final.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# final.fit_generator(train_datagen.flow(images_train,ages_train,
#                              batch_size = batch_size),
#                           epochs=epochs,
#                           steps_per_epoch=images_train.shape[0] // batch_size,
#         #                   verbose=True,
#                           validation_data=(images_test, ages_test),
#                           callbacks=[reduce_learning_rate,
#                                      early_stopping])

In [ ]:
final.evaluate(images_test, ages_test)

In [ ]:
idx = 48
plt.imshow(images_test[idx])
print(ages_test[idx])

In [ ]:
y = final.predict(images_test[50:51])
print(y)

In [ ]:
final.save('model.h5')